## About
This is a notebook for calculating the following statistics in a map:
- producer's accuracies and user's accuracies by class,
- overall accuracy, and
- area estimates. 

Each statistic is accompanied by a 95% confidence interval. 
To do this, we follow the notation and calculations in Olofsson et al..

The data input needs to be:
1. a csv of the points assessed with two columns: map_class and ref_class. Map class is the classification of the point in the map, ref_class is the "ground truth" classification of the point. 

2. a csv with the number of pixels per class in the map.

## Reference

Pontus Olofsson, Giles M. Foody, Martin Herold, Stephen V. Stehman, Curtis E. Woodcock, Michael A. Wulder, Good practices for estimating area and assessing accuracy of land change, Remote Sensing of Environment,Volume 148, 2014, Pages 42-57, ISSN 0034-4257, https://doi.org/10.1016/j.rse.2014.02.015.  (https://www.sciencedirect.com/science/article/pii/S0034425714000704)

In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

# Assuming repository's parent directory is the home directory
home = os.path.expanduser("~")
os.chdir(os.path.join(home,'iceplant-detection-santa-barbara'))

### Load Data

To run the example in Olofsson et al., do not import any data and do not execute the cells in this section. 
Instead, go to the next section and execute the cell having the Olofsson et al. data directly.

In [2]:
prefix = 'final_model'
validation_data_dir = os.path.join(os.getcwd(),
                              'data',
                              'map',
                              'validation_data')
# load csv with validation points
df = pd.read_csv(os.path.join(validation_data_dir,
                              prefix+'_map_and_reference_classes.csv'))

# specify number of classes in the map
n_classes = 3

#load counts of pixels per class in map
pixel_count_path = os.path.join(validation_data_dir,
                              'pixel_counts',
                                prefix+'_combined_pixel_counts_total.csv')
pix_counts = pd.read_csv(pixel_count_path)
pix_counts = pix_counts.to_numpy()[0]
pix_counts

array([120173466,   5981423, 188071487])

In [3]:
# counts by reference class
print('Points in each reference class')
print(np.unique(df.ref_class, return_counts=True), '\n')

# counts by map class: these should match the counts given by the stratified sample design
print('Points in each map class')
print(np.unique(df.map_class, return_counts=True))

Points in each reference class
(array([0, 1, 2]), array([321, 264, 404])) 

Points in each map class
(array([0, 1, 2]), array([289, 296, 404]))


### Confusion Matrix

Here we create a confusion matrix $n$ such that 

$n_{i,j}$ = number of points predicted as $i$, known to be $j$, 

which is equivalent to

$n_{i,j}$ = number of points that have map class as $i$ and reference class $j$.

In [4]:
# CALCULATE CONFUSION MATRIX FOR IMPORTED DATA

# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
# using confusion_matrix directly we get a matrix C such that
# C_{i,j} = known to be i, predicted as  j 
# The notation in the paper is 
# n_{i,j} = predicted as i, known to be j 
# so we need to take the transpose

n = confusion_matrix(df.ref_class, df.map_class, labels=range(n_classes)).T
n

array([[247,   2,  40],
       [ 26, 262,   8],
       [ 48,   0, 356]])

### User's Accuracy

In [5]:
# -------------------------------------
# points in sample that had class i in map (predicted as i, any true class j)
# these will also be used in overal accuracy and producer's accuracies
n_idot = [sum(n[i,:]) for i in range(n_classes)]

# -------------------------------------
# estimated users' accuracy (precision for each class: TP/(TP+FP))
U_hat = [n[i,i] / n_idot[i] for i in range(n_classes)]

var_U_hat = [U_hat[i] * (1-U_hat[i])/(n_idot[i]-1) for i in range(n_classes)]

# -------------------------------------
print("user's accuracies:", [x*100 for x in U_hat])
print("user's accuracies confidence interval:", 1.95*np.sqrt(var_U_hat)*100)

user's accuracies: [85.46712802768167, 88.51351351351352, 88.11881188118812]
user's accuracies confidence interval: [4.04961416 3.62011191 3.14301418]


### Overal Accuracy

In [6]:
# total number of pixels in the map
total_pix = sum(pix_counts)

# list with the fractions of area in map mapped as each class
W = [pix_counts[i]/ total_pix for i in range(n_classes)]      

# -------------------------------------
# overall accuracy
O_hat = sum([W[i]*n[i,i]/n_idot[i] for i in range(n_classes)])
print('overall accuracy:', O_hat*100)

# -------------------------------------
var_O_hat = sum([ W[i]**2 * U_hat[i] * (1-U_hat[i])/(n_idot[i]-1) for i in range(n_classes)])

# std error of estimated overall accuracy
print('overall accuracy confidence interval:', 1.95*np.sqrt(var_O_hat)*100, '\n')

overall accuracy: 87.11220904276064
overall accuracy confidence interval: 2.4376493210335597 



### Producer's Accuracy

In [7]:
p_hat_dotj = []
# estimated producer's accuracy (sensitiviy for each class TP/(TP+FN))
P_hat = []  

for j in range(n_classes):
    # list of p_hat_ij with fixed j
    p_hat_ij = [ W[i]*n[i,j]/n_idot[i] for i in range(n_classes) ]
    p_hat_dotj.append(sum(p_hat_ij))  # equation (9)


P_hat= [ (W[j]*n[j,j]/n_idot[j]) / p_hat_dotj[j] for j in range(n_classes)]
# -------------------------------------
print("producer's accuracies:", [x*100 for x in P_hat])

# -------------------------------------
# -------------------------------------
# VARIANCE
# notice N_jdot is pixel_counts[j]
N_hat_cdotj = []
for j in range(n_classes):
    summands = [ pix_counts[i] * n[i,j]/n_idot[i] for i in range(n_classes)]
    N_hat_cdotj.append(sum(summands))

# -------------------------------------
summand1 = [ (pix_counts[j]**2) * ((1-P_hat[j])**2) * U_hat[j] * (1-U_hat[j]) / (n_idot[j] - 1) 
            for j in range(n_classes)]

# -------------------------------------
summand2 = []
for j in range(n_classes):
    inner = []
    for i in range(n_classes):
        if i!=j:
            inner.append( (pix_counts[i]**2) /(n_idot[i]-1) * (n[i,j])/(n_idot[i]) * ( 1 - n[i,j]/n_idot[i]) ) 
    summand2.append((P_hat[j]**2) * sum(inner))

# -------------------------------------
var_P_hat = [1/(N_hat_cdotj[j]**2) *  (summand1[j] + summand2[j]) for j in range(n_classes)]

# -------------------------------------
# -------------------------------------
print("producer's accuracies confidence interval:", 1.95*np.sqrt(var_P_hat)*100, '\n')

producer's accuracies: [81.78798853172489, 86.42429238701436, 90.79850351390348]
producer's accuracies confidence interval: [ 3.91598154 16.15675927  2.39143534] 



### Area Estimates

In [8]:
# PERCENTAGE OF AREA ESTIMATION
# we had calculated the area estimators before, they are used in producer's accuracy
print("percentage of area per class: \n", [x*100 for x in p_hat_dotj])

# -------------------------------------
# STD ERROR
SE_p_hat_dotj = []
for j in range(n_classes):
    summands = [ (W[i]**2) * (n[i,j]/n_idot[i]) * (1 -  (n[i,j]/n_idot[i]))/ (n_idot[i]-1) 
                for i in range(n_classes)]
    SE_p_hat_dotj.append(np.sqrt(sum(summands)))
    
print("confidence interval for percentage area per class:\n", [x*1.96*100 for x in SE_p_hat_dotj])


percentage of area per class: 
 [39.96460579533045, 1.9495556092790984, 58.08583859539045]
confidence interval for percentage area per class:
 [2.449942581976087, 0.37266421030865493, 2.4296033021275183]


In [9]:

map_area = total_pix * (21158/235086)

approx_area_per_class = [map_area * p_hat_dotj[i] for i in range(n_classes)]
print("approx area per class (ha): \n", approx_area_per_class)

SE_area_per_class = [map_area * SE_p_hat_dotj[i] for i in range(n_classes)]
print("confidence interval for area per class (m^2):\n", [x*1.96 for x in SE_area_per_class])

approx area per class (ha): 
 [11302278.810610583, 551348.3897653435, 16427094.167157656]
confidence interval for area per class (m^2):
 [692861.4352732152, 105392.12695391006, 687109.3402110785]


In [10]:
# in km^2, assuming a resolution of 0.5m per pixel side
map_area = total_pix * 0.36 / (1000**2)

approx_area_per_class = [map_area * p_hat_dotj[i] for i in range(n_classes)]
print("approx area per class (km^2): \n", approx_area_per_class)

SE_area_per_class = [map_area * SE_p_hat_dotj[i] for i in range(n_classes)]
print("confidence interval for area per class (km^2):\n", [x*1.96 for x in SE_area_per_class])

approx area per class (km^2): 
 [45.20855969040702, 2.205366458091275, 65.70756921150169]
confidence interval for area per class (km^2):
 [2.771411684192743, 0.4215633273726857, 2.748403586682587]
